<a href="https://colab.research.google.com/github/marcos-code/Machine-Learning/blob/main/Cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A filosofia que você deve ter na cabeça é que **o primeiro contato de um modelo de machine learning com dados de teste, deve ocorrer apenas na última etapa do processo.**

Dentre todos os fatores que nos levam a chegar a essa conclusão, estão os fatos que usar datasets de testes para ajustes, levarão a um provável **overfitting** e carregarão um *bias* (viés).

Assim, para que você possa aprender os parâmetros de um modelo e preservar os dados de teste, mostrarei a técnica de **cross-validation**, seguindo o fluxograma da documentação oficial do `sklearn`.

<center><img alt="Colaboratory logo" width="40%" src="https://raw.githubusercontent.com/carlosfab/dsnp2/master/img/grid_search_workflow.png"></center>

Em tempo, veremos em outra aula como determinar os parâmetros do modelo usando uma técnica conhecida como `grid search`.

In [10]:
# importar os pacotes necessários
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import numpy as np

# garantir replicabilidade
np.random.seed(42)

# importar o arquivo
df = pd.read_csv("http://dl.dropboxusercontent.com/s/6d91j46mkcdj4qv/heart-disease-clean.csv?dl=1")

#Escolha do modelo

In [23]:
# 1. Escolher e importar um modelo
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn import metrics

# 2. Instanciar e escolher os hyperparameters
model = LogisticRegression()


#3. Separar os dados entre feature matrix e target vector
X = df.drop('num', axis=1)
y= df['num']

# 3.1 Dividir o dataset entre o treino e teste
X_train, X_test, y_train, y_test = train_test_split(X,y)

# 3.2 Padronizar os dados de treino
scaler = StandardScaler().fit(X_train)
X_train_transformed = scaler.transform(X_train)

# 4. Cross-Validation
from sklearn.pipeline import make_pipeline
model = make_pipeline(StandardScaler(), LogisticRegression())
scores = cross_val_score(model, X_train, y_train, cv=5)

print("scores: ", scores)
print("Acurácia: %0.2f (+/- %.02f)" % (scores.mean(), scores.std() * 2))




scores:  [0.82608696 0.86956522 0.77777778 0.66666667 0.91111111]
Acurácia: 0.81 (+/- 0.17)


Acima, você pode ver o score médio e o intervalo de confiança de 95%.

Por padrão, o score é computado de acordo com o método de score do próprio estimador. No entanto, é possível escolher outra métrica mais alinhada com a realidade do seu dataset.

Para conhecer as métricas possíveis, [acesse este link](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter).

In [28]:
scores = cross_val_score(model, X_train_transformed, y_train, cv=5, scoring="f1")
print("scores: ", scores)

scores:  [0.8        0.85       0.72222222 0.65116279 0.9       ]


Aqui eu quero chamar a atenção para uma coisa. Repare que no primeiro modelo, fizemos a padronização do nosso `X_train_transformed = scaler.transform(X_train)` e somente após usamos a técnica de cross-validation.

Vou explicar o motivo dessa não ser uma prática adequada. Veja na imagem abaixo que a cada rodada do nosso k-fold (técnica básica de cross-validation), o subconjunto em azul é deixado como "teste". 

<center><img alt="Colaboratory logo" width="40%" src="https://raw.githubusercontent.com/carlosfab/dsnp2/master/img/grid_search_cross_validation.png"></center>

Isso não é ideal, pois queremos simular uma condição onde nosso modelo de machine learning nunca tenha visto esse subconjunto.

Essa etapa de padronização ou qualquer outra de pré-processamento, pode ser estruturada com um `pipeline`.

In [32]:
from sklearn.pipeline import make_pipeline
model = make_pipeline(StandardScaler(), LogisticRegression())
scores = cross_val_score(model, X_train, y_train, cv=5)

print("Scores: ", scores)
print("Acurácia: %0.2f(+/-%0.2f)" % (scores.mean(), scores.std() * 2))

Scores:  [0.82608696 0.86956522 0.77777778 0.66666667 0.91111111]
Acurácia: 0.81(+/-0.17)
